# Assignment - Supervised Learning flow

# Part 1(a) Student details:
* Please write the First-Name, First letter of Last-Name and last 4 digits of the i.d. for each student. 

In [ ]:
# stundent details example: John S. 9812
#                       Tomer B 9317: 
# (if exists)           student details 2: 
# (if exists)           student details 3: 
# (if exists)           student details 4: 

## Part 1(b) - Chat-GPT/other AI-agent/other assistance used:
* If you changed the prompt until you got a satisfying answer, please add all versions
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add information:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>   
AI agent name: ChatGPT
Goal: visual clarity and efficiency
Propmpt1:
    
Propmpt2:
    
Propmpt3: 


AI agent name 2:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 

Other assistanse:    
</pre>

## Part 1(c) - Learning Problem and dataset explaination.
* Please explain in one paragraph
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add explaining text:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>
# Titanic - Supervised Learning Flow

מחברת זו ממלאת את מבנה המטלה:

- הקדמה וקבצי סטודנט
- טעינה ו-EDA
- Feature engineering
- Grid search עם 5-fold CV על LogisticRegression ו-RandomForest
- בחירה של המודל הטוב ביותר והערכה על holdout test

המחברת נכתבה אוטומטית על ידי סקריפט תחזיק/י עריכה במידה ונדרש.
    
</pre>

## Part 2 - Initial Preparations 
You could add as many code cells as needed

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Load dataset (uploaded file 'titanic_test.csv')
df = pd.read_csv('/mnt/data/titanic_test.csv')
df.head()

In [ ]:
# Basic info
print('Shape:', df.shape)
print('\nColumns:', df.columns.tolist())
print('\nMissing values:\n', df.isna().sum())

## Part 3 - Experiments
You could add as many code cells as needed

In [ ]:
# Quick EDA plots
plt.figure(figsize=(6,4))
plt.hist(df['Age'].dropna(), bins=15)
plt.title('Age distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(6,4))
plt.hist(df['Fare'].dropna(), bins=15)
plt.title('Fare distribution')
plt.xlabel('Fare')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(6,4))
plt.bar([0,1], df['Survived'].value_counts().loc[[0,1]].values)
plt.title('Survival counts (0=dead,1=survived)')
plt.xlabel('Survived')
plt.ylabel('Count')
plt.xticks([0,1])
plt.show()

pclass_rates = df.groupby('Pclass')['Survived'].mean()
plt.figure(figsize=(6,4))
plt.bar(pclass_rates.index.astype(str), pclass_rates.values)
plt.title('Survival rate by Pclass')
plt.xlabel('Pclass')
plt.ylabel('Survival rate')
plt.show()

sex_rates = df.groupby('Sex')['Survived'].mean()
plt.figure(figsize=(6,4))
plt.bar(sex_rates.index.astype(str), sex_rates.values)
plt.title('Survival rate by Sex')
plt.xlabel('Sex')
plt.ylabel('Survival rate')
plt.show()

In [ ]:
# Feature engineering
df['FamilySize'] = df['SibSp'] + df['Parch']
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','FamilySize']
target = 'Survived'
X = df[features].copy()
y = df[target].copy()

display(X.head())

## Part 4 - Training 
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 3)

In [ ]:
# Train/Test split (stratified 80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)
print('\nTrain target distribution:\n', y_train.value_counts())
print('\nTest target distribution:\n', y_test.value_counts())

## Part 5 - Apply on test and show model performance estimation

In [ ]:
# Preprocessing pipelines
numeric_features = ['Age','Fare','SibSp','Parch','FamilySize','Pclass']
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])

categorical_features = ['Sex','Embarked']
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [ ]:
# Define pipelines and parameter grids
pipe_lr = Pipeline(steps=[('pre', preprocessor), ('clf', LogisticRegression(max_iter=200))])
param_grid_lr = {'clf__C': [0.1, 1, 10], 'clf__penalty':['l2']}

pipe_rf = Pipeline(steps=[('pre', preprocessor), ('clf', RandomForestClassifier(random_state=42))])
param_grid_rf = {'clf__n_estimators': [50,100], 'clf__max_depth': [None,5,10]}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs_lr = GridSearchCV(pipe_lr, param_grid_lr, scoring='f1', cv=cv, n_jobs=1, return_train_score=True)
gs_rf = GridSearchCV(pipe_rf, param_grid_rf, scoring='f1', cv=cv, n_jobs=1, return_train_score=True)

print('Fitting Logistic Regression grid...')
gs_lr.fit(X_train, y_train)
print('Best LR:', gs_lr.best_params_, 'CV F1:', gs_lr.best_score_)

print('\nFitting Random Forest grid...')
gs_rf.fit(X_train, y_train)
print('Best RF:', gs_rf.best_params_, 'CV F1:', gs_rf.best_score_)

In [ ]:
# Choose best and evaluate on holdout test
if gs_lr.best_score_ >= gs_rf.best_score_:
    best_model = gs_lr.best_estimator_
    best_name = 'LogisticRegression'
    best_cv = gs_lr.best_score_
else:
    best_model = gs_rf.best_estimator_
    best_name = 'RandomForest'
    best_cv = gs_rf.best_score_

print(f'Selected model: {best_name} (CV F1={best_cv:.4f})')

y_pred = best_model.predict(X_test)
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred)
}
print('\nTest metrics:', metrics)
print('\nClassification report:\n', classification_report(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

# Show first 5 predictions
preds = X_test.reset_index(drop=True).copy()
preds['true_Survived'] = y_test.reset_index(drop=True)
preds['pred_Survived'] = y_pred
display(preds.head(5))

In [ ]:
# (Optional) Save best model to disk
import joblib
joblib.dump(best_model, '/mnt/data/titanic_best_model.joblib')